In [1]:
%run Scaffold_Y_lipo.ipynb

In [2]:
model.compartments

{'e': 'extracellular space',
 'c': 'cytosol',
 'm': 'mitochondria',
 'x': 'peroxisome/glyoxysome',
 'n': 'nucleus',
 'g': 'golgi apparatus',
 'v': 'vacuole',
 'r': 'endoplasmic reticulum'}

In [3]:
print(model.objective)

Maximize
1.0*BIOMASS_SC4_bal - 1.0*BIOMASS_SC4_bal_reverse_bb385


In [4]:
medium = model.medium

medium['EX_so4_e'] = 1000
medium['EX_pi_e'] = 1000
medium['EX_o2_e'] = 1000
medium['EX_nh4_e'] = 1000
medium['EX_glc__D_e'] = 300
medium['EX_h2o_e'] = 1000

model.medium = medium
model.medium


{'EX_nh4_e': 1000,
 'EX_o2_e': 1000,
 'EX_pi_e': 1000,
 'EX_so4_e': 1000,
 'EX_glc__D_e': 300,
 'EX_h2o_e': 1000}

Testing the model with its original objective

In [5]:
medium = model.medium
with model:
    medium['EX_glc__D_e'] = 200
    model.medium = medium
    EPA_production = model.optimize().objective_value
    print('Maximum productivity =', EPA_production, 'mmol/gDW*h')
    maximum_yield = EPA_production / (-1*(model.reactions.get_by_id('EX_glc__D_e').flux))
    print('Maximum theoretical yield =', maximum_yield, 'mmol-EPA/mmol-glc')
    print('Objective of model is =', model.objective)

Maximum productivity = 19.46467518075362 mmol/gDW*h
Maximum theoretical yield = 0.0973233759037681 mmol-EPA/mmol-glc
Objective of model is = Maximize
1.0*BIOMASS_SC4_bal - 1.0*BIOMASS_SC4_bal_reverse_bb385


Testing the model with the objective of EPA

In [6]:
model.objective = model.reactions.DM_Eicosapentaenoic_Acid
medium = model.medium
with model:
    medium['EX_glc__D_e'] = 200
    model.medium = medium
    EPA_production = model.optimize().objective_value
    print('Maximum productivity =', EPA_production, 'mmol/gDW*h')
    maximum_yield = EPA_production / (-1*(model.reactions.get_by_id('EX_glc__D_e').flux))
    print('Maximum theoretical yield =', maximum_yield, 'mmol-EPA/mmol-glc')
    print('Objective of model is =', model.objective)


Maximum productivity = 31.800467653936046 mmol/gDW*h
Maximum theoretical yield = 0.15900233826968024 mmol-EPA/mmol-glc
Objective of model is = Maximize
1.0*DM_Eicosapentaenoic_Acid - 1.0*DM_Eicosapentaenoic_Acid_reverse_e7faa


In [7]:
model.reactions.DM_Eicosapentaenoic_Acid

Reaction identifier,DM_Eicosapentaenoic_Acid
Name,EPA demand
Memory address,0x15be45c90
Stoichiometry,Eicosapentaenoic_Acid --> EPA -->
GPR,
Lower bound,0
Upper bound,1000.0


### Lets try to optimize the medium

According to [https://onlinelibrary.wiley.com/doi/epdf/10.1002/ejlt.200700224], the optimal medium for *L. starkeyi* is: 
mixed  sugar 73.3 g/L  (glucose  48.9 g/L,  xylose  24.4 g/L),  yeast  extract 7.9 g/L, FeSO 44.0 mg/L, KH2PO 47.0 g/L, Na2HPO 42.0 g/L, MgSO4-7H2O 1.0 g/L, and NH4Cl 0.5 g/L.

In [8]:
OPTmedium = model.medium

OPTmedium["r_1718"] = 100 #Xylose exchange
OPTmedium["r_1714"] = 100 #Glucose exchange
OPTmedium["r_2020"] = 1 #Potassium exchange
OPTmedium["r_1861"] = 1#Fe2+
OPTmedium["r_2049"] = 1 #Sodium
OPTmedium["r_1654"] = 1 #Ammonium
OPTmedium["r_1832"] = 10 #H+
OPTmedium["r_1992"] = 10 #Oxygen
OPTmedium["r_2005"] = 10 #Phosphate
OPTmedium["r_2060"] = 10 #Sulphate
OPTmedium["r_2100"] = 5 #H20
OPTmedium["r_4593"] = 10 #Chloride
OPTmedium["r_4594"] = 10 #Cu2+
OPTmedium["r_4595"] = 10 #Mn2+
OPTmedium["r_4596"] = 10 #Zn2+
OPTmedium["r_4597"] = 10 #Mg2+
OPTmedium["r_4600"] = 10 #Ca2+

# Maybe some amino acids should also be considered - All have exchange reactions in the model

model.medium = OPTmedium

model.objective = model.reactions.DM_Eicosapentaenoic_Acid
medium = model.medium
with model:
    medium['r_1714'] = 20
    model.medium = medium
    EPA_production = model.optimize().objective_value
    print('Maximum productivity =', EPA_production, 'mmol/gDW*h')
    maximum_yield = EPA_production / (-1*(model.reactions.get_by_id('r_1714').flux))
    print('Maximum theoretical yield =', maximum_yield, 'mmol-EPA/mmol-glc')
    print('Objective of model is =', model.objective)

KeyError: 'r_1718'

In [ ]:
model.objective = model.reactions.get_by_id('r_2111')
solution = model.optimize()
print('Maximum theoretical biomass productivity:', solution.fluxes['r_2111'], '/h') 

model.objective = model.reactions.DM_Eicosapentaenoic_Acid
EPA_prod = model.optimize().objective_value
print('Maximum theoretical productivity of EPA:', EPA_prod, 'mmol/gDW*h')

In [ ]:
solution.fluxes['r_2111']

## Find the optimal medium for the model by setting bounds and optimizing the model

In [ ]:
model.reactions.get_by_id('r_2111')